In [ ]:
#general definitions and imports
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import logging
logging.Logger.disabled=True

import os
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import numpy as np
import matplotlib.pyplot as plt

import ipytest
ipytest.autoconfig()
from pytest import approx

from pathlib import Path

from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Spice.Netlist import Circuit
from PySpice.Probe.Plot import plot
from PySpice.Math import *

import schemdraw
import schemdraw.elements as elm
from schemdraw import dsp
from schemdraw import logic

## setup spice library path
directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)
## set the project directory as directory_path
directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
# definitions
        
class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4, name='voltage_divider'):
        self.__name__ = name
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class JackIn(SubCircuitFactory):
    __name__ = 'jack_in'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=1@u_kΩ, name='jack_in', value='DC 5'):
        self.__name__ = name
        super().__init__()
        self.V(1, 'n2', self.gnd, value)

class Vref(SubCircuitFactory):
    __name__ = 'vref'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=1@u_kΩ, name='vref', value='DC 5'):
        self.__name__ = name
        super().__init__()
        self.V(1, 'n2', self.gnd, value)

class JackOut(SubCircuitFactory):
    __name__ = 'jack_out'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, name='jack_out'):
        self.__name__ = name
        super().__init__()
        self.R(1, 'n2', 'n1', R)

class PatchRef(SubCircuitFactory):
    __name__ = 'patch'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, name='patch'):
        self.__name__ = name
        super().__init__()
        self.R(1, 'n1', 'n2', 0)

class PatchGnd(SubCircuitFactory):
    __name__ = 'patch'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, name='patch'):
        self.__name__ = name
        super().__init__()
        self.R(1, 'n1', 'n3', 0)


In [ ]:

%%run_pytest[clean] --json=../../build/steckfeld-test.json

# define the tests

def test_input():
    kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
    parser = SpiceParser(path=str(kicad_netlist_path))

    circuit = parser.build_circuit(ground=0)
    circuit.include(spice_library['TL072c'])

    circuit.V('1', '+15V', circuit.gnd, 'DC 15')
    circuit.V('2', '-15V', circuit.gnd, 'DC -15')

    for c in ( VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_1'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_2'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_3'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_4'), 
            JackIn(name='IN_1', value='AC 5 DC 5 SINE(0 5 1k)'), 
            JackIn(name='IN_2', value='AC 0 DC 0'), 
            JackIn(name='IN_3', value='AC 5 DC 5 SINE(0 5 3k)'), 
            JackIn(name='IN_4', value='AC 5 DC 5 SINE(0 5 4k)'), 
            JackOut(name='OUT_1'), 
            JackOut(name='OUT_2'),
            Vref(name='Vref1', value='DC 5'),
            Vref(name='Vref2', value='DC 5'),
            PatchGnd(name='patch1'),
            PatchGnd(name='patch2') ):
        circuit.subcircuit(c)

    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    analysis  = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=20@u_ms)

    values = np.array(analysis["IN_1"])
    result_min = np.amin(values)
    result_max = np.amax(values)
    result_avg = np.average(values) 

    print( 'result: %f, %f, %f' % (result_max, result_min, result_avg))
    assert result_max == approx( 5, rel=1e-2 )
    assert result_min == approx( -5, rel=1e-2 )
    assert result_avg == approx( 0.00035970155344531863, rel=1e-2 )


def test_input_both():
    kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
    parser = SpiceParser(path=str(kicad_netlist_path))

    circuit = parser.build_circuit(ground=0)
    circuit.include(spice_library['TL072c'])

    circuit.V('1', '+15V', circuit.gnd, 'DC 15')
    circuit.V('2', '-15V', circuit.gnd, 'DC -15')

    for c in ( VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_1'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_2'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_3'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_4'), 
            JackIn(name='IN_1', value='AC 5 DC 5 SINE(0 5 1k)'), 
            JackIn(name='IN_2', value='AC 0 DC 5 SINE(0 5 1k)'), 
            JackIn(name='IN_3', value='AC 5 DC 5 SINE(0 5 3k)'), 
            JackIn(name='IN_4', value='AC 5 DC 5 SINE(0 5 4k)'), 
            JackOut(name='OUT_1'), 
            JackOut(name='OUT_2'),
            Vref(name='Vref1', value='DC 5'),
            Vref(name='Vref2', value='DC 5'),
            PatchGnd(name='patch1'),
            PatchGnd(name='patch2') ):
        circuit.subcircuit(c)

    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    analysis  = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=20@u_ms)

    values = np.array(analysis["IN_1"])
    result_min = np.amin(values)
    result_max = np.amax(values)
    result_avg = np.average(values) 
    print( 'result: %f, %f, %f' % (result_max, result_min, result_avg))
    assert result_max == approx( 10, rel=1e-2 )
    assert result_min == approx( -10, rel=1e-2 )
    assert result_avg == approx( 0.0006928803365574028, rel=1e-2 )


def test_output():
    kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
    parser = SpiceParser(path=str(kicad_netlist_path))

    circuit = parser.build_circuit(ground=0)
    circuit.include(spice_library['TL072c'])

    circuit.V('1', '+15V', circuit.gnd, 'DC 15')
    circuit.V('2', '-15V', circuit.gnd, 'DC -15')
    circuit.V('3', 'OUT_1a', circuit.gnd, 'AC 5 DC 5 SINE(0 5 1k)')
    circuit.V('4', 'OUT_1b', circuit.gnd, 'AC 5 DC 0')

    for c in ( VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_1'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_2'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_3'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_4'), 
            JackIn(name='IN_1', value='AC 5 DC 5 SINE(0 5 1k)'), 
            JackIn(name='IN_2', value='AC 0 DC 5 SINE(0 5 1k)'), 
            JackIn(name='IN_3', value='AC 5 DC 5 SINE(0 5 3k)'), 
            JackIn(name='IN_4', value='AC 5 DC 5 SINE(0 5 4k)'), 
            JackOut(name='OUT_1'), 
            JackOut(name='OUT_2'),
            Vref(name='Vref1', value='DC 5'),
            Vref(name='Vref2', value='DC 5'),
            PatchGnd(name='patch1'),
            PatchGnd(name='patch2') ):
        circuit.subcircuit(c)

    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    analysis  = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=20@u_ms)

    values = np.array(analysis["Net-_C11-Pad2_"])
    result_min = np.amin(values)
    result_max = np.amax(values)
    result_avg = np.average(values) 
    print( 'result output: %f, %f, %f' % (result_max, result_min, result_avg))
    assert result_max == approx( 5, rel=1e-2 )
    assert result_min == approx( -5, rel=1e-2 )
    assert result_avg == approx( 0.00016442563947679473, rel=1e-2 )

def test_level_shift():
    kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
    parser = SpiceParser(path=str(kicad_netlist_path))

    circuit = parser.build_circuit(ground=0)
    circuit.include(spice_library['TL072c'])

    circuit.V('1', '+15V', circuit.gnd, 'DC 15')
    circuit.V('2', '-15V', circuit.gnd, 'DC -15')
    circuit.V('3', 'OUT_1a', circuit.gnd, 'AC 5 DC 5 SINE(5 5 1k)')
    circuit.V('4', 'OUT_1b', circuit.gnd, 'AC 5 DC 0')

    for c in ( VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_1'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_2'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_3'), 
            VoltageDivider(R=100@u_kΩ, w=0.00000000000001, name='voltage_divider_4'), 
            JackIn(name='IN_1', value='AC 5 DC 5 SINE(0 5 1k)'), 
            JackIn(name='IN_2', value='AC 0 DC 5 SINE(0 5 1k)'), 
            JackIn(name='IN_3', value='AC 5 DC 5 SINE(0 5 3k)'), 
            JackIn(name='IN_4', value='AC 5 DC 5 SINE(0 5 4k)'), 
            JackOut(name='OUT_1'), 
            JackOut(name='OUT_2'),
            Vref(name='Vref1', value='DC 1.68'),
            Vref(name='Vref2', value='DC -5'),
            PatchRef(name='patch1'),
            PatchGnd(name='patch2') ):
        circuit.subcircuit(c)

    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    analysis  = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=20@u_ms)

    values = np.array(analysis["Net-_C11-Pad2_"])
    result_min = np.amin(values)
    result_max = np.amax(values)
    result_avg = np.average(values) 
    print( 'result: %f, %f, %f' % (result_max, result_min, result_avg))
    assert result_max == approx( 5.039814, rel=1e-2 )
    assert result_min == approx( -4.960018, rel=1e-2 )
    assert result_avg == approx( 0.00016442764132744227, rel=1e-2 )



# construction

the function depends on the patching and settings of the potentiometers. an attenuverter is used in different modules like the  Serge VCS, Maths and also in the polivoks filter. while this module can be used as a simple mixer you can also adjust cv signals. such a signal can be simply attenuated or inverted. when feeding in a positive singal slope (for example from an adsr). the signal will be turned to a neagative dc singal. if the inpout signal is ac coupled the signal will be inverted. the function is the setting on the potentiometer of the channel. when the potentiometer is at center position. the output signal is attenuated to zero. no output singal is present. when turning to the right. the signal will be stronger until the output signal is equal to the input signal. when turning the potentiomenter to the left, the output signal will be phase shifted by 180°. when you turn the potentiometer to full left the output signal will be the inverted input signal. all the inputs are mixed to the output signal; unless you connect the output of a channel. then this output does not accure on the mixed signal.
the attenuverter is build around a differential amplifier with an opamp. 
{: class="mb-6"} 


In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=1.1)

JI4 = d.add(elm.AudioJack(xy=[0, d.unit], lftlabel='$IN_4$'))
d.add(elm.Ground("right", xy=JI4.sleeve))
d.add(elm.Line( "right", l=d.unit, at=JI4.tip))
D4 = d.add(dsp.Amp())
d.add(elm.Line( "right", l=d.unit))
JO4 = d.add(elm.AudioJack("left", switch=True, flip=True, anchor='tip', rgtlabel='$OUT_4$'))
d.add(elm.Ground("right", xy=JO4.sleeve))

JI3 = d.add(elm.AudioJack(xy=[0, 2*d.unit], lftlabel='$IN_3$'))
d.add(elm.Ground("right", xy=JI3.sleeve))
d.add(elm.Line( "right", l=d.unit, at=JI3.tip))
D3 = d.add(dsp.Amp())
d.add(elm.Line( "right", l=d.unit))
JO3 = d.add(elm.AudioJack("left", switch=True, flip=True, anchor='tip', rgtlabel='$OUT_3$'))
d.add(elm.Ground("right", xy=JO3.sleeve))

JI2 = d.add(elm.AudioJack(xy=[0, 3*d.unit], lftlabel='$IN_2$'))
d.add(elm.Ground("right", xy=JI2.sleeve))
d.add(elm.Line( "right", l=d.unit, at=JI2.tip))
D2 = d.add(dsp.Amp())
d.add(elm.Line( "right", l=d.unit))
JO2 = d.add(elm.AudioJack("left", switch=True, flip=True, anchor='tip', rgtlabel='$OUT_2$'))
d.add(elm.Ground("right", xy=JO2.sleeve))

JI1 = d.add(elm.AudioJack(xy=[0, 4*d.unit], lftlabel='$IN_1$'))
d.add(elm.Ground("right", xy=JI1.sleeve))
d.add(elm.Line( "right", l=d.unit, at=JI1.tip))
D1 = d.add(dsp.Amp())
d.add(elm.Line( "right", l=d.unit))
JO1 = d.add(elm.AudioJack("left", switch=True, flip=True, anchor='tip', rgtlabel='$OUT_1$'))
d.add(elm.Ground("right", xy=JO1.sleeve))

d.add(elm.Line("left", l=d.unit/2, xy=JO1.tipswitch))
d.add(elm.Line("down"))

d.add(elm.Line("left", l=d.unit/2, xy=JO2.tipswitch))
d.add(elm.Dot())
d.add(elm.Line("down"))

d.add(elm.Line("left", l=d.unit/2, xy=JO3.tipswitch))
d.add(elm.Dot())
d.add(elm.Line("down"))

d.add(elm.Line("left", l=d.unit/2, xy=JO4.tipswitch))
d.add(elm.Dot())
d.add(elm.Line("down", l=d.unit/2))
S = d.add(dsp.Sum())  

d.add(elm.Line( "down", xy=S.E, l=d.unit/2))
d.add(elm.Line( "right", l=d.unit/2))
JO = d.add(elm.AudioJack("left", anchor="tip", flip=True, rgtlabel='$OUT$'))
d.add(elm.Ground("right", xy=JO.sleeve))

d.draw()


In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=0.9)

O = d.add(elm.Opamp( label='$U_1$'))

d.add(elm.Line( "left", xy=O.in1, l=d.unit/8))
R1 = d.add(elm.Resistor( "left", label='$R_1$'))
d.add(logic.Dot())    
R2 = d.add(elm.Resistor( "down", label='$R_3$'))
d.add(elm.Resistor( "down", label='$R_4$'))
d.add(elm.Ground)

L1 = d.add(elm.Line('left', l=d.unit/2, xy=R1.end))
d.add(elm.Line('down', l=d.unit/2))
RV = d.add(elm.Potentiometer('down', toplabel='$UV_1$'))
d.add(elm.Line('down', l=d.unit/2))
d.add(elm.Ground)

d.add(elm.Line('left', l=d.unit/4, xy=L1.end, lftlabel='$IN$'))

d.add(elm.Line('right', tox=R2.end, xy=RV.tap))
d.add(logic.Dot())

d.add(elm.Line('right', l=d.unit/2, xy=R2.end))
d.add(elm.Line('up', l=d.unit*0.58))
d.add(elm.Line('right', l=d.unit*0.63))

d.add(logic.Dot(xy=R1.start))
d.add(elm.Line('up', l=d.unit))
d.add(elm.Resistor('right', label='$R_2$', tox=O.out+d.unit/8))
d.add(elm.Line('down', toy=O.out))
d.add(logic.Dot())

d.add(elm.Line('right', l=d.unit/4, xy=O.out, rgtlabel='$OUT$'))

d.draw()

the circuit of the attenuverter is based on the opamp dfferential amplifer. the two resistors at the non-inverting inputs are replaced with a potentiometer. the potentiometer is configured as a voltage divider and replace R3 and R4 from the differential amplifier. when the position of the potentiometer is adjusted, the output voltage will change or invert. at the center position  the output is zero volrs. the response to the potentiometer is linear. we can make it an centered s-curve by adding two parallel resistors (R5,R6) to the potentiometer [[2][2]].

a simulation with the resistance at RV 1 changed from 0-100%. first run is the linear result. the second run is done with the resistors R5 and R6 added. This results in a s curve. the shape of the curve can be changed with the values of the resistors.
{: class="mb-6"} 


In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<div class="hero is-fullheight mb-4 schema" style="background: url('{{ '/assets/steckfeld-board.jpg' | relative_url }}') no-repeat center center; background-size: cover; background-attachment: fixed;">
   <div class="hero-body is-fullheight">
        <div class="content">
        </div>
    </div>
</div>
'''))


# bom

{% include bom.html %}
{% include callouts.html %}
{% include reports.html%}

# usage

the input jacks are wired to 5 volts when nothing is connected.

all channels are mixed to the out jack. when something is connected to the channel out, this channel is removed from the overall mix.

_mixer_
* connect the different channels from audio or cv sources to the in jacks.
* connect the out jack to something
* turn the pots clockwise to adjust the volume.

_attenuverter_
* connect all or a single channel. 
* when you turn the pot knob counter clockwise the signal is inverted.
{: class="mb-6"}




# references

- [Op Amp Differential Amplifier](http://www.ecircuitcenter.com/Circuits/opdif/opdif.htm) eCircuit  Center
- [Beginners' Guide to Potentiometers](https://sound-au.com/pots.htm)  Rod Elliott (ESP)
{: class="mb-6"}